In [1]:
# importing necessary libraries and modules

import math
import random
import requests
import numpy as np
import pandas as pd
from pathlib import Path
from zipfile import ZipFile
from requests.auth import HTTPBasicAuth
from ipynb.fs.full.File_Variables import *
from ipynb.fs.full.EB_runtimes_main import *
from ipynb.fs.full.WB_runtimes_main import *
from ipynb.fs.full.Database_Connections import *
from ipynb.fs.full.Get_WB_trip_opportunities import *
from datetime import datetime, timedelta, date, time

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', 1)

In [2]:
# retreiving the Metra GTFS Feeds

def getpublished(username, password):

    r = requests.get('https://gtfsapi.metrarail.com/gtfs/raw/published.txt', 
                     auth = HTTPBasicAuth(username, password), verify = False)
    
    timestamp = r.content.decode('UTF-8')
    
    path = Path('./published.txt')
    if path.is_file():
        file = open("published.txt","r")
        previous_timestamp = file.readline()
        if previous_timestamp == timestamp:
            print("Previous published data is same as the current published data! No need to download the GTFS feeds again!") 
            
        elif previous_timestamp != timestamp:
            print("Updating the published data and getting the latest GTFS feeds!!")
            published_file = open("published.txt", "w")
            published_file.write(timestamp)
            published_file.close()
            zipfile = requests.get('https://gtfsapi.metrarail.com/gtfs/raw/schedule.zip', 
                                   auth = HTTPBasicAuth(username, password), verify = False)
            with open("schedule.zip", "wb") as zip_file:
                zip_file.write(zipfile.content)
            with ZipFile("./schedule.zip", 'r') as zip_object:
                zip_object.extractall("./schedule")
            
        else:
            pass
        
    else:
        print("Getting the latest published data!!")
        published_file = open("published.txt", "w")
        published_file.write(timestamp)
        published_file.close()
        zipfile = requests.get('https://gtfsapi.metrarail.com/gtfs/raw/schedule.zip', 
                               auth = HTTPBasicAuth(username, password), verify = False)
        with open("schedule.zip", "wb") as zip_file:
            zip_file.write(zipfile.content)
        with ZipFile("./schedule.zip", 'r') as zip_object:
            zip_object.extractall("./schedule")
        
getpublished('89349b017d9747c9160cd9e48c2d9284', 'b01fd672a633e381b1e24e75e216df77')

Previous published data is same as the current published data! No need to download the GTFS feeds again!


C:\Users\MBhutada.int\AppData\Local\anaconda3\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'gtfsapi.metrarail.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [3]:
# reading the retrieved Metra GTFS Feed files as dataframe

agency = pd.read_csv("./schedule/agency.txt")
calendar = pd.read_csv("./schedule/calendar.txt")
calendar_dates = pd.read_csv("./schedule/calendar_dates.txt")
fare_attributes = pd.read_csv("./schedule/fare_attributes.txt")
fare_rules = pd.read_csv("./schedule/fare_rules.txt")
routes = pd.read_csv("./schedule/routes.txt")
shapes = pd.read_csv("./schedule/shapes.txt")
stop_times = pd.read_csv("./schedule/stop_times.txt")
stops = pd.read_csv("./schedule/stops.txt")
trips = pd.read_csv("./schedule/trips.txt")

In [4]:
# converting the 'start_date' and 'end_date' columns in calendar dataframe in the proper format (ex. 20230531 is converted to 2023-05-31)

calendar.columns = calendar.columns.str.strip()
stop_times.columns = stop_times.columns.str.strip()
trips.columns = trips.columns.str.strip()

df_list = [calendar, stop_times, trips]

for i in df_list:
    column_names = i.select_dtypes(include = "object")
    for j in column_names:
        i[j] = i[j].str.strip()

calendar['start_date'] = pd.to_datetime(calendar['start_date'], format='%Y%m%d')
calendar['end_date'] = pd.to_datetime(calendar['end_date'], format='%Y%m%d')

calendar['start_date'] = calendar['start_date'].apply(lambda x: x.date())
calendar['end_date'] = calendar['end_date'].apply(lambda x: x.date())

In [5]:
# joining stop_times, trips, calendar dataframes using inner join

intermediate_merge = pd.merge(stop_times, trips, on = 'trip_id', how = 'inner')
joined_data = pd.merge(intermediate_merge, calendar, on = 'service_id', how = 'inner')
joined_data = joined_data[(joined_data['route_id'] == "UP-NW") & (joined_data['trip_headsign'] == "Chicago OTC") & (joined_data['stop_id'] == "IRVINGPK")]
joined_data = joined_data[['service_id', 'start_date', 'arrival_time', 'stop_id', 'route_id', 'monday', 'tuesday', 'wednesday', 'thursday', 'friday', 'saturday', 'sunday', 'end_date']]

In [6]:
wrigley_field_data = get_data()

In [7]:
# defining the holiday list in the current year

# holiday_list = [datetime.strptime('2023-01-02', "%Y-%m-%d").date(), 
#                 datetime.strptime('2023-05-29', "%Y-%m-%d").date(), 
#                 datetime.strptime('2023-07-04', "%Y-%m-%d").date(), 
#                 datetime.strptime('2023-09-04', "%Y-%m-%d").date(), 
#                 datetime.strptime('2023-11-23', "%Y-%m-%d").date(), 
#                 datetime.strptime('2023-12-25', "%Y-%m-%d").date()]

In [8]:
# converting DTE column from MM/DD/YYYY to YYYY-MM-DD

wrigley_field_data['DTE'] = pd.to_datetime(wrigley_field_data['DTE'], format='%m/%d/%Y')
wrigley_field_data['DTE'] = wrigley_field_data['DTE'].apply(lambda x: x.date())

In [9]:
#resetting the index of the wrigley_field_data dataframe

wrigley_field_data = wrigley_field_data.reset_index()
wrigley_field_data = wrigley_field_data.drop(['index'], axis = 1)

# getting the correct daytype (weekday, saturday or sunday) for each date

wrigley_field_data['DAY'] = wrigley_field_data['DTE'].apply(lambda x: x.strftime("%A"))
wrigley_field_data['DAY'] = wrigley_field_data['DTE'].apply(lambda x: "Sunday" if x in holiday_list else x.strftime("%A"))
wrigley_field_data['DAY'] = wrigley_field_data['DAY'].apply(lambda x: "Weekday" if x in ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday"] else "Saturday" if x == "Saturday" else "Sunday")

# differentiating between Cubs and Non-Cubs events

wrigley_field_data['EVENT_CATEGORY'] = wrigley_field_data['EVENT_TYPE'].apply(lambda x: "Cubs" if x in ['Cubs Day', 'Cubs Night'] else "Non-Cubs")

In [10]:
# removing rows with null values in 'START_TIME' and 'END_TIME' columns in wrigley_field_data and then resetting the index

wrigley_field_data = wrigley_field_data[(wrigley_field_data['START_TIME'].notnull()) & (wrigley_field_data['END_TIME'].notnull())]
wrigley_field_data = wrigley_field_data.reset_index()
wrigley_field_data = wrigley_field_data.drop(['index'], axis = 1)

In [11]:
# function to get the Metra service ids operational during an event date based on the event daytype

def get_service_id(date):
    
    if date in holiday_list:
        weekday = "sunday"
    else:
        weekday = date.strftime("%A").lower()
    return calendar[(calendar['start_date'] <= date) & (calendar['end_date'] >= date) & (calendar[weekday])]

In [12]:
# retrieving the service_id for every date in the wrigley_field_data dataframe, final results stored in service_id_for_event_dates list

service_id_dataframes_for_every_DTE = []
service_id_for_event_dates = [] # metra service ids operational during an event date

for i in wrigley_field_data['DTE'].tolist():
    service_id_dataframes_for_every_DTE.append(get_service_id(i))
    
for i in service_id_dataframes_for_every_DTE:
    service_id_for_event_dates.append(i['service_id'].tolist())

In [13]:
# retrieving the Metra arrival times for every event/event date

service_ids_arrival_times_for_every_event = [] # List of metra train arrival times for every event date

for service_ids in service_id_for_event_dates:
    if len(service_ids) > 1:
        arrival_times_for_event_with_multiple_service_id = pd.Series() #defining a Series to store arrival times for event dates with multiple opertional service ids
        for service_id in service_ids:
            arrival_times_for_event_with_multiple_service_id = arrival_times_for_event_with_multiple_service_id._append(joined_data[joined_data['service_id'] == service_id]['arrival_time'],
                                                                                                                        ignore_index = True)
        service_ids_arrival_times_for_every_event.append(arrival_times_for_event_with_multiple_service_id.tolist())
    else:
        for service_id in service_ids:
            service_ids_arrival_times_for_every_event.append(joined_data[joined_data['service_id'] == service_id]['arrival_time'].tolist())

# filtering the Metra arrival times for every event/event date by only considering the Metra arrival times that arrive before 24:00:00            
            
for i, j in zip(service_ids_arrival_times_for_every_event, range(len(service_ids_arrival_times_for_every_event))):
    service_ids_arrival_times_for_every_event[j] = [m for m in i if m <= '23:59:59']

In [14]:
# storing the Metra arrival times for every event/event date into the wrigley_field_data dataframe by define a new column 'METRA_ARRIVAL_TIMES'

wrigley_field_data['METRA_ARRIVAL_TIMES'] = service_ids_arrival_times_for_every_event

# converting the Metra arrival times for each event/event date from str to time object

for i in wrigley_field_data['METRA_ARRIVAL_TIMES']:
    for j in range(len(i)):
        i[j] = datetime.strptime(i[j], '%H:%M:%S').time()

In [15]:
# converting the event start time for each event/event date from str to time object

wrigley_field_data['START_TIME'] = wrigley_field_data['START_TIME'].apply(lambda x: datetime.strptime(x, '%I:%M:%S %p').time())

In [16]:
# taking only the distinct Metra arrival times for every event/event date -- removing duplicate entries of Metra arrival times for every event/event date

wrigley_field_data['METRA_ARRIVAL_TIMES'] = wrigley_field_data['METRA_ARRIVAL_TIMES'].apply(lambda x: list(set(x)))

In [17]:
# defining the start_time_window for each event/event date for the blocking process/creating the blocks

date = date(1, 1, 1)

def get_start_time_window(row):
    
    return (datetime.combine(date, row) - timedelta(minutes = 210)).time()

wrigley_field_data['START_TIME_WINDOW'] = wrigley_field_data['START_TIME'].apply(get_start_time_window)

In [18]:
# selecting all the Metra arrival times that are greater than the event START_TIME_WINDOW and less than the event actual START_TIME

metra_arrival_times = []
for i in range(len(wrigley_field_data)):
    revised_metra_arrival_times_for_every_event = []
    for j in wrigley_field_data['METRA_ARRIVAL_TIMES'][i]:
        if ((wrigley_field_data['START_TIME_WINDOW'][i] <= j) & (wrigley_field_data['START_TIME'][i] >= j)):
            revised_metra_arrival_times_for_every_event.append(j)
        else:
            pass
    metra_arrival_times.append(revised_metra_arrival_times_for_every_event)

wrigley_field_data['METRA_ARRIVAL_TIMES'] = metra_arrival_times
wrigley_field_data['METRA_ARRIVAL_TIMES'] = wrigley_field_data['METRA_ARRIVAL_TIMES'].apply(lambda x: tuple(x))

In [19]:
# creating grouped dataframes based on 'EVENT_CATEGORY', 'DAY', 'START_TIME' and 'METRA_ARRIVAL_TIMES' columns and storing those grouped dataframes in individual_grouped_df_list list

grouped_df_list = []
for i in wrigley_field_data.groupby(['EVENT_CATEGORY', 'DAY', 'START_TIME', 'METRA_ARRIVAL_TIMES']):
    grouped_df_list.append(i)
    
individual_grouped_df_list = []
grouped_dataframes = wrigley_field_data.groupby(['EVENT_CATEGORY', 'DAY', 'START_TIME', 'METRA_ARRIVAL_TIMES'])
for i in range(len(grouped_df_list)):
    individual_grouped_df_list.append(grouped_dataframes.get_group(grouped_df_list[i][0]))

In [20]:
# fetching the EB runtimes for all 3 daytypes

EB_runtimes_weekday = get_the_EB_runtimes_df(rtver_df['RTVER_NO'][0])
EB_runtimes_saturday = get_the_EB_runtimes_df(rtver_df['RTVER_NO'][1])
EB_runtimes_sunday = get_the_EB_runtimes_df(rtver_df['RTVER_NO'][2])

In [21]:
# fetching the WB runtimes for all 3 daytypes

WB_runtimes_weekday = get_the_WB_runtimes_df(rtver_df['RTVER_NO'][0])
WB_runtimes_saturday = get_the_WB_runtimes_df(rtver_df['RTVER_NO'][1])
WB_runtimes_sunday = get_the_WB_runtimes_df(rtver_df['RTVER_NO'][2])

In [22]:
# defining two empty dictionaries 

d = {}
d_main = {}

for i in range(len(individual_grouped_df_list)):
    d[i] = pd.DataFrame(columns=['Trip', 'Place', 'Time'])

In [23]:
# getting the start times for EB trips from IrvBlu for all the Metra arrival times for each event in each group based on their daytype 
# we add 10 minutes to each Metra arrival times for each event in each group based on their daytype so that passengers get times to get to the bus stop after they alight the metra trains

for i, j in zip(individual_grouped_df_list, range(len(individual_grouped_df_list))):
    for k, c in zip(i["METRA_ARRIVAL_TIMES"], range(len(i['METRA_ARRIVAL_TIMES']))):
        for m in k:
            if i['DAY'].iloc[c] == 'Weekday':
                d[j] = pd.concat([d[j], EB_runtimes_weekday[(EB_runtimes_weekday['Place'] == 'IrvBlu') & (EB_runtimes_weekday['Time'] == (datetime.combine(datetime.today(), m) + timedelta(minutes = 10)).time())]])
            elif i['DAY'].iloc[c] == 'Saturday':
                d[j] = pd.concat([d[j], EB_runtimes_saturday[(EB_runtimes_saturday['Place'] == 'IrvBlu') & (EB_runtimes_saturday['Time'] == (datetime.combine(datetime.today(), m) + timedelta(minutes = 10)).time())]])
            else:
                d[j] = pd.concat([d[j], EB_runtimes_sunday[(EB_runtimes_sunday['Place'] == 'IrvBlu') & (EB_runtimes_sunday['Time'] == (datetime.combine(datetime.today(), m) + timedelta(minutes = 10)).time())]])

In [24]:
# retrieving the entire EB trips from IrvCum to IrvBwy for each Metra arrival time for each each event in each group based on the daytype 
# the results are stored in the dictionary d for each group

for i, m in zip(individual_grouped_df_list, range(len(d))):
    if i['DAY'].iloc[0] == 'Weekday':
        d[m] = EB_runtimes_weekday[EB_runtimes_weekday['Trip'].isin(d[m]['Trip'].tolist())]
    elif i['DAY'].iloc[0] == 'Saturday':
        d[m] = EB_runtimes_saturday[EB_runtimes_saturday['Trip'].isin(d[m]['Trip'].tolist())]
    else:
        d[m] = EB_runtimes_sunday[EB_runtimes_sunday['Trip'].isin(d[m]['Trip'].tolist())]

In [25]:
# storing the results from dictionary d to d_main

for i in d:
    d_main[i] = d[i]

# retrieving only the timepoints from IrvBlu to IrvBwy for each EB trip for each Metra arrival time for each event in each group     
    
for i in d:
    d[i] = d[i][d[i]['Place'].isin(['IrvBlu', 'IrvKim', 'IrvWes', 'IrvAsh', 'SheLs', 'IrvBwy'])]

In [26]:
# getting the EB trip start time for each Metra arrival time for each event in each group from IrvBlu

trip_start_times_after_metra_arrival = {}

for i in d:
    trip_start_times_after_metra_arrival[i] = d[i][d[i]['Place'] == 'IrvBlu']['Time'].tolist()

In [27]:
# calculating the minimum layover time for each EB trip from IrvBlu

layover_dict = {}

for i in d_main:
    d_main[i] = d_main[i][(d_main[i]['Place'] == 'IrvCum') | (d_main[i]['Place'] == 'IrvBwy')]

    layover_times = []
    
    for j in d_main[i]['Trip'].unique():
        trip_start_end_time = d_main[i][d_main[i]['Trip'] == j]['Time']
        started_time = trip_start_end_time.iloc[0]
        ended_time = trip_start_end_time.iloc[1]
        
        datetime1 = datetime.combine(date, started_time)
        datetime2 = datetime.combine(date, ended_time)
        
        total_trip_time = (datetime2 - datetime1).seconds/60
        layover_time = 0.1 * total_trip_time
        
        layover_times.append(layover_time)
        
    layover_dict[i] = layover_times

In [28]:
# rounding off the minimum layovers for each EB trip to correct one decimal place

for i in d_main:
    rounded_layover_times = []
    for j in layover_dict[i]:
        if j % 1 >= 0.00 and j % 1 <= 0.50:
            j = math.floor(j)
            j = j + 0.5
            rounded_layover_times.append(j)
        elif j % 1 > 0.50 and j % 1 < 1.0:
            j = math.ceil(j)
            rounded_layover_times.append(j)
        else:
            pass
        
    layover_dict[i] = rounded_layover_times

In [29]:
# now we will construct a time window at IrvBwy to start the WB trip from IrvBwy. 
# after defining the window, we look at the WB trip opportunities from IrvBwy and select the most earliest WB trip opportunity from IrvBwy that falls within the window we define
# see code below

In [30]:
# getting the WB trip start time window at IrvBwy by adding minimum layover to the EB trip end time at IrvBwy

WB_trip_start_time_window = {}

for i in d_main:
    WB_trip_start_time_window_per_table = []
    
    for j, k in zip(d_main[i][d_main[i]['Place'] == 'IrvBwy']['Time'], layover_dict[i]):
        WB_trip_start_time_window_per_table.append(((datetime.combine(date, j) + timedelta(minutes = k)).time()))
    
    WB_trip_start_time_window[i] = WB_trip_start_time_window_per_table

In [31]:
# getting the WB trip end time window at IrvBwy by adding 30 minutes to the EB trip end time at IrvBwy

WB_trip_end_time_window = {}

for i in d_main:
    WB_trip_end_time_window_per_table = []
    
    for j in d_main[i][d_main[i]['Place'] == 'IrvBwy']['Time']:
        WB_trip_end_time_window_per_table.append(((datetime.combine(date, j) + timedelta(minutes = 30)).time()))
    
    WB_trip_end_time_window[i] = WB_trip_end_time_window_per_table

In [32]:
# retrieving the WB trip opportunities for each daytype respectively

WB_trip_opportunities_weekday = get_wb_trip_opportunities(vscver_df['VSCVER_ID'][0])
WB_trip_opportunities_saturday = get_wb_trip_opportunities(vscver_df['VSCVER_ID'][1])
WB_trip_opportunities_sunday = get_wb_trip_opportunities(vscver_df['VSCVER_ID'][2])

In [33]:
# converting the WB trip opportunities for each daytype into a dataframe

WB_trip_opportunities_weekday_df = pd.DataFrame(WB_trip_opportunities_weekday, columns=['WB_WK_TPO'])
WB_trip_opportunities_saturday_df = pd.DataFrame(WB_trip_opportunities_saturday, columns=['WB_SA_TPO'])
WB_trip_opportunities_sunday_df = pd.DataFrame(WB_trip_opportunities_sunday, columns=['WB_SU_TPO'])

In [34]:
# retrieving the actual WB trip start times from IrvBwy for each EB trip in each group based on their respective daytype

wb_potential_trip_start_times = {}
for i, j in zip(individual_grouped_df_list, d_main):
    if i['DAY'].iloc[0] == 'Weekday':
        WB_potential_trip_start_time_wk = []
        for b, c in zip(WB_trip_start_time_window[j], WB_trip_end_time_window[j]):
            wk = WB_trip_opportunities_weekday_df[(WB_trip_opportunities_weekday_df['WB_WK_TPO'] >= b) & (WB_trip_opportunities_weekday_df['WB_WK_TPO'] < c)]
            wk = sorted(wk['WB_WK_TPO'].tolist())[0]
            WB_potential_trip_start_time_wk.append(wk)
        
        wb_potential_trip_start_times[j] = WB_potential_trip_start_time_wk
        
    elif i['DAY'].iloc[0] == 'Saturday':
        WB_potential_trip_start_time_sa = []
        for b, c in zip(WB_trip_start_time_window[j], WB_trip_end_time_window[j]):
            sa = WB_trip_opportunities_saturday_df[(WB_trip_opportunities_saturday_df['WB_SA_TPO'] >= b) & (WB_trip_opportunities_saturday_df['WB_SA_TPO'] < c)]
            sa = sorted(sa['WB_SA_TPO'].tolist())[0]
            WB_potential_trip_start_time_sa.append(sa)
        
        wb_potential_trip_start_times[j] = WB_potential_trip_start_time_sa
            
    else:
        WB_potential_trip_start_time_su = []
        for b, c in zip(WB_trip_start_time_window[j], WB_trip_end_time_window[j]):
            su = WB_trip_opportunities_sunday_df[(WB_trip_opportunities_sunday_df['WB_SU_TPO'] >= b) & (WB_trip_opportunities_sunday_df['WB_SU_TPO'] < c)]
            su = sorted(su['WB_SU_TPO'].tolist())[0]
            WB_potential_trip_start_time_su.append(su)
        
        wb_potential_trip_start_times[j] = WB_potential_trip_start_time_su

In [35]:
# retrieving the actual WB trip end times at IrvBlu for each WB trip in each group based on their daytype

wb_potential_trip_end_times = {}
for i in range(len(individual_grouped_df_list)):
    if individual_grouped_df_list[i]['DAY'].iloc[0] == 'Saturday':
        wb_trip_end_times_at_blu = []
        for j in wb_potential_trip_start_times[i]:
            # getting the WB endtimes at IrvBlu for each grouped table -- saturday daytype
            wb_trip_end_times_at_blu.append(WB_runtimes_saturday[(WB_runtimes_saturday['Trip'] == WB_runtimes_saturday[(WB_runtimes_saturday['Place'] == 'IrvBwy') & (WB_runtimes_saturday['Time'] == j)]['Trip'].iloc[0]) & (WB_runtimes_saturday['Place'] == 'IrvBlu')]['Time'].iloc[0])
        wb_potential_trip_end_times[i] = wb_trip_end_times_at_blu
        
    elif individual_grouped_df_list[i]['DAY'].iloc[0] == 'Weekday':
        wb_trip_end_times_at_blu = []
        for j in wb_potential_trip_start_times[i]:
            # getting the WB endtimes at IrvBlu for each grouped table -- weekday daytype
            wb_trip_end_times_at_blu.append(WB_runtimes_weekday[(WB_runtimes_weekday['Trip'] == WB_runtimes_weekday[(WB_runtimes_weekday['Place'] == 'IrvBwy') & (WB_runtimes_weekday['Time'] == j)]['Trip'].iloc[0]) & (WB_runtimes_weekday['Place'] == 'IrvBlu')]['Time'].iloc[0])
        wb_potential_trip_end_times[i] = wb_trip_end_times_at_blu
        
    else:
        wb_trip_end_times_at_blu = []
        for j in wb_potential_trip_start_times[i]:
            # getting the WB endtimes at IrvBlu for each grouped table -- sunday daytype
            wb_trip_end_times_at_blu.append(WB_runtimes_sunday[(WB_runtimes_sunday['Trip'] == WB_runtimes_sunday[(WB_runtimes_sunday['Place'] == 'IrvBwy') & (WB_runtimes_sunday['Time'] == j)]['Trip'].iloc[0]) & (WB_runtimes_sunday['Place'] == 'IrvBlu')]['Time'].iloc[0])
        wb_potential_trip_end_times[i] = wb_trip_end_times_at_blu

In [36]:
# adding minimum layover = 4 minutes to actual WB trip end times at IrvBlu

for i in wb_potential_trip_end_times:
    for j, k in zip(wb_potential_trip_end_times[i], range(len(wb_potential_trip_end_times[i]))):
        j = (datetime.combine(date, j) + timedelta(minutes = 4)).time()
        wb_potential_trip_end_times[i][k] = j

In [37]:
# calculating the actual layover at IrvBwy

actual_layover_EB_to_WB = {}
for x in d_main:
    layovers = []
    for i, j in zip(d_main[x][d_main[x]['Place'] == 'IrvBwy']['Time'], wb_potential_trip_start_times[x]):
        actual_layover = datetime.combine(date, j) - datetime.combine(date, i)
        layovers.append(actual_layover.seconds/60)
    actual_layover_EB_to_WB[x] = layovers

In [38]:
# defining a dictionary called sub_blocks that contains the times at which the buses are ready to start in the next direction for each group.
# Here is the general representation of its structure. 
# EB1: time at which the bus is ready to start in the EB direction from IrvBlu based on the Metra arrival times
# WB1: revised time/actual time at which the bus is ready to start from IrvBwy to IrvBlu in the WB direction 
# EB2: revised time/actual time at which the bus is ready to start in the EB direction from IrvBlu 

# these times will be used to create the final blocks (actual blocks) for each group.

sub_blocks = {}
for a in trip_start_times_after_metra_arrival:
    blocks = {}
    for i in range(len(trip_start_times_after_metra_arrival[a])):
        blocking = []
        for m in trip_start_times_after_metra_arrival[a]:
            if (datetime.combine(date, wb_potential_trip_end_times[a][i]) - datetime.combine(date, m)).seconds/60 <= actual_layover_EB_to_WB[a][i]:
                minutes_to_minus_from_wb_potential_trip_start_times = (datetime.combine(date, wb_potential_trip_end_times[a][i]) - datetime.combine(date, m)).seconds/60
                wb_potential_trip_start_times[a][i] = (datetime.combine(date, wb_potential_trip_start_times[a][i]) - timedelta(minutes = minutes_to_minus_from_wb_potential_trip_start_times)).time()
                wb_potential_trip_end_times[a][i] = m
            else:
                pass
        blocking.append(trip_start_times_after_metra_arrival[a][i])
        blocking.append(wb_potential_trip_start_times[a][i])
        blocking.append(wb_potential_trip_end_times[a][i])
        blocks[i] = blocking
    sub_blocks[a] = blocks

In [39]:
final_blocks_for_each_group = {}

for x in sub_blocks:
    sample_df = pd.DataFrame(sub_blocks[x])
    sample_df = sample_df.T.rename(columns = {0: 'W-E-1', 1: 'E-W-1', 2: 'W-E-2'})
    
    sample_ddf = pd.DataFrame(columns=['W-E-1', 'E-W-1', 'W-E-2', 'E-W-2', 'W-E-3'])
    for i in sample_df['W-E-2']:
        lst = sorted(sample_df[sample_df['W-E-1'] >= i]['W-E-1'].tolist())
        nlst = []
        if len(lst) > 0:
            nlst.append(sample_df[sample_df['W-E-2'] == i].iloc[0].tolist())
            nlst.append(sample_df[sample_df['W-E-1'] == lst[0]].iloc[0].tolist())
            nlst = sum(nlst, [])
        else:
            nlst = (sample_df[sample_df['W-E-2'] == i].iloc[0].tolist()) 
            nlst.append(np.nan)
            nlst.append(np.nan)

        if len(nlst) != 0 and np.nan not in nlst:
            nlst.pop(2)
            sample_ddf.loc[len(sample_ddf)] = nlst
        else:
            sample_ddf.loc[len(sample_ddf)] = nlst 
    
    for q in sample_ddf['W-E-1']:
        if q in sample_ddf['W-E-2'].tolist() and np.nan in sample_ddf[sample_ddf['W-E-1'] == q].iloc[0].tolist():
            sample_ddf.drop(sample_ddf[sample_ddf['W-E-1'] == q].index, axis = 0, inplace = True)
        
        else:
            pass
        
    sample_ddf.reset_index(inplace = True)
    sample_ddf.drop(['index'], axis = 1, inplace = True)
    
    sample_dddf = pd.DataFrame(columns=['W-E-1', 'E-W-1', 'W-E-2', 'E-W-2', 'W-E-3', 'E-W-3', 'W-E-4'])

    if len(sample_ddf) % 2 == 1 and len(sample_ddf) == 3:
        for i in sample_ddf['W-E-2']:
            nlst = []
            lst = sample_ddf[sample_ddf['W-E-1'] == i]['W-E-1'].tolist()

            if len(lst) == 1:
                nlst.append(sample_ddf[sample_ddf['W-E-2'] == i].iloc[0].tolist())
                nlst.append(sample_ddf[sample_ddf['W-E-1'] == lst[0]].iloc[0].tolist())
                nlst = sum(nlst, [])
                nlst.pop(2)
                nlst.pop(2)
                nlst.pop(2)
                sample_dddf.loc[len(sample_dddf)] = nlst
                sample_ddf.drop(sample_ddf.index[sample_ddf['W-E-1'] == lst[0]], axis = 0, inplace = True)

            elif len(lst) == 0 and i in sample_ddf['W-E-2'].values:
                sample_dddf = sample_dddf._append(pd.Series(sample_ddf[sample_ddf['W-E-2'] == i].iloc[0].tolist(), index = sample_dddf.columns[:len(sample_ddf[sample_ddf['W-E-2'] == i].iloc[0].tolist())]), ignore_index=True)

            else:
                pass

    elif len(sample_ddf) == 2:
        pass
    
    if len(sample_dddf) == 0:
        final_blocks = sample_ddf
    
    else:
        final_blocks = sample_dddf
        
    final_blocks_for_each_group[x] = final_blocks

In [40]:
staging_time_limit_after_event_start_time_at_wrigley_field = []

for i in range(len(individual_grouped_df_list)):
    staging_time_limit_after_event_start_time_at_wrigley_field.append((datetime.combine(date, individual_grouped_df_list[i]['START_TIME'].iloc[0]) + timedelta(minutes = 40)).time())

In [41]:
actual_block_times_dictionary_for_each_group = {}

for i, j in zip(final_blocks_for_each_group, individual_grouped_df_list):
    
    if j['DAY'].iloc[0] == 'Saturday':
        
        actual_block_times = {}
        
        for a, b in final_blocks_for_each_group[i].iterrows():
            
            times_for_each_block = []
            b = b.tolist()
            
            for x in b:
                
                if (b.index(x) % 2 == 0) and x != None: 
                    
                    ref_df = EB_runtimes_saturday[EB_runtimes_saturday['Trip'] == EB_runtimes_saturday[(EB_runtimes_saturday['Time'] == x) & (EB_runtimes_saturday['Place'] == 'IrvBlu')]['Trip'].iloc[0]]
                    arrive_time_at_SheLs = ref_df[ref_df['Place'] == 'SheLs']['Time'].iloc[0]
                    staging_time_at_WF = (datetime.combine(date, arrive_time_at_SheLs) + timedelta(minutes = 4)).time()
                    
                    if staging_time_at_WF <= staging_time_limit_after_event_start_time_at_wrigley_field[i]:
                        times_for_each_block.append(x)
                        actual_block_times[a] = times_for_each_block
                    
                    else:
                        break
                
                elif x != None:
                    times_for_each_block.append(x)
                    actual_block_times[a] = times_for_each_block
                    previous_staging_time_at_WF = staging_time_at_WF
                    
                else:
                    pass
                    
            if len(actual_block_times[a]) % 2 == 0:
                del actual_block_times[a][-1]
                actual_block_times[a].append(previous_staging_time_at_WF)
                
            else:
                actual_block_times[a].append(staging_time_at_WF)
            
            actual_block_times_dictionary_for_each_group[i] = actual_block_times    
            
    if j['DAY'].iloc[0] == 'Sunday':
        
        actual_block_times = {}
        
        for a, b in final_blocks_for_each_group[i].iterrows():

            times_for_each_block = []
            b = b.tolist()
            
            for x in b:
                
                if (b.index(x) % 2 == 0) and x != None: 
                    
                    ref_df = EB_runtimes_sunday[EB_runtimes_sunday['Trip'] == EB_runtimes_sunday[(EB_runtimes_sunday['Time'] == x) & (EB_runtimes_sunday['Place'] == 'IrvBlu')]['Trip'].iloc[0]]
                    arrive_time_at_SheLs = ref_df[ref_df['Place'] == 'SheLs']['Time'].iloc[0]
                    staging_time_at_WF = (datetime.combine(date, arrive_time_at_SheLs) + timedelta(minutes = 4)).time()
                    
                    if staging_time_at_WF <= staging_time_limit_after_event_start_time_at_wrigley_field[i]:
                        times_for_each_block.append(x)
                        actual_block_times[a] = times_for_each_block
                    
                    else:
                        break
                
                elif x != None:
                    times_for_each_block.append(x)
                    actual_block_times[a] = times_for_each_block
                    previous_staging_time_at_WF = staging_time_at_WF
                    
                else:
                    pass
                
            if len(actual_block_times[a]) % 2 == 0:
                del actual_block_times[a][-1]
                actual_block_times[a].append(previous_staging_time_at_WF)
                
            else:
                actual_block_times[a].append(staging_time_at_WF)
            
            actual_block_times_dictionary_for_each_group[i] = actual_block_times 
            
    if j['DAY'].iloc[0] == 'Weekday':
        
        actual_block_times = {}
        
        for a, b in final_blocks_for_each_group[i].iterrows():

            times_for_each_block = []
            b = b.tolist()
            
            for x in b:
                
                if (b.index(x) % 2 == 0) and x != None: 
                    
                    ref_df = EB_runtimes_weekday[EB_runtimes_weekday['Trip'] == EB_runtimes_weekday[(EB_runtimes_weekday['Time'] == x) & (EB_runtimes_weekday['Place'] == 'IrvBlu')]['Trip'].iloc[0]]
                    arrive_time_at_SheLs = ref_df[ref_df['Place'] == 'SheLs']['Time'].iloc[0]
                    staging_time_at_WF = (datetime.combine(date, arrive_time_at_SheLs) + timedelta(minutes = 4)).time()
                    
                    if staging_time_at_WF <= staging_time_limit_after_event_start_time_at_wrigley_field[i]:
                        times_for_each_block.append(x)
                        actual_block_times[a] = times_for_each_block
                    
                    else:
                        break
                
                elif x != None:
                    times_for_each_block.append(x)
                    actual_block_times[a] = times_for_each_block
                    previous_staging_time_at_WF = staging_time_at_WF
                    
                else:
                    pass
                    
            if len(actual_block_times[a]) % 2 == 0:
                del actual_block_times[a][-1]
                actual_block_times[a].append(previous_staging_time_at_WF)
                
            else:
                actual_block_times[a].append(staging_time_at_WF)
            
            actual_block_times_dictionary_for_each_group[i] = actual_block_times 

In [42]:
actual_blocks_df = pd.DataFrame(actual_block_times_dictionary_for_each_group)

In [43]:
for i, j in actual_blocks_df.iterrows():
    for a in j:
        for b in a:
            if a.index(b) == 0:
                pull_out_time = (datetime.combine(date, b) - timedelta(minutes = 24)).time()
                a.append(pull_out_time)
                a.sort()
                
            else:
                break

In [44]:
EB_distances_and_tp_notes = {
                                 'IrvBlu': ['0.000', ''],
                                 'IrvKim': ['0.903', ''],
                                 'IrvWes': ['1.252', ''],
                                 'IrvAsh': ['0.988', ''],
                                 'SheLs' : ['1.345', 'EARLY'],
                                 'IrvBwy': ['0.420', '']
                            }


WB_distances_and_tp_notes = {
                                 'IrvBwy': ['0.000', ''],
                                 'IrvAsh': ['0.891', ''],
                                 'IrvWes': ['0.992', ''],
                                 'IrvKim': ['1.261', ''],
                                 'IrvBlu': ['0.894', 'CBIrBluA']
                            }


trip_notes_list = []

for i in individual_grouped_df_list:
    
    if i['EVENT_CATEGORY'].iloc[0] == 'Cubs' and str(i['START_TIME'].iloc[0]) =='12:05:00':
        trip_notes_list.append('CSICubPY')
        
    elif i['EVENT_CATEGORY'].iloc[0] == 'Cubs' and str(i['START_TIME'].iloc[0]) =='12:20:00':
        trip_notes_list.append('CSICubAA')
        
    elif i['EVENT_CATEGORY'].iloc[0] == 'Cubs' and str(i['START_TIME'].iloc[0]) =='13:20:00':
        trip_notes_list.append('CSICubsA')
        
    elif i['EVENT_CATEGORY'].iloc[0] == 'Cubs' and str(i['START_TIME'].iloc[0]) =='15:05:00':
        trip_notes_list.append('CSI1935')
        
    elif i['EVENT_CATEGORY'].iloc[0] == 'Cubs' and str(i['START_TIME'].iloc[0]) =='18:05:00':
        trip_notes_list.append('CSICubST')
        
    elif i['EVENT_CATEGORY'].iloc[0] == 'Cubs' and str(i['START_TIME'].iloc[0]) =='18:15:00':
        trip_notes_list.append('CSICubFX')
        
    elif i['EVENT_CATEGORY'].iloc[0] == 'Cubs' and str(i['START_TIME'].iloc[0]) =='18:40:00':
        trip_notes_list.append('CSICubC')
    
    elif i['EVENT_CATEGORY'].iloc[0] == 'Cubs' and str(i['START_TIME'].iloc[0]) =='19:05:00':
        trip_notes_list.append('CSICubPW')
        
    elif i['EVENT_CATEGORY'].iloc[0] == 'Non-Cubs':
        trip_notes_list.append('CSIatWF')

In [45]:
num_of_saturday_events = 0
num_of_sunday_events = 0
num_of_weekday_events = 0

for x in range(len(individual_grouped_df_list)):
    if individual_grouped_df_list[x]['DAY'].iloc[0] == 'Saturday':
        num_of_saturday_events = num_of_saturday_events + 1
        
    elif individual_grouped_df_list[x]['DAY'].iloc[0] == 'Sunday':
        num_of_sunday_events = num_of_sunday_events + 1
        
    else:
        num_of_weekday_events = num_of_weekday_events + 1
        

if num_of_saturday_events == 1:
    for x in range(len(individual_grouped_df_list)):
        if individual_grouped_df_list[x]['DAY'].iloc[0] == 'Saturday':
            saturday_event_time = str(individual_grouped_df_list[x]['START_TIME'].iloc[0])
            saturday_event_time = saturday_event_time.replace(':', '')[:-2]
            
        else:
            pass
else:
    saturday_event_time = ''


if num_of_sunday_events == 1:
    for x in range(len(individual_grouped_df_list)):
        if individual_grouped_df_list[x]['DAY'].iloc[0] == 'Sunday':
            sunday_event_time = str(individual_grouped_df_list[x]['START_TIME'].iloc[0])
            sunday_event_time = sunday_event_time.replace(':', '')[:-2]
            
        else:
            pass
else:
    sunday_event_time = ''


if num_of_weekday_events == 1:
    for x in range(len(individual_grouped_df_list)):
        if individual_grouped_df_list[x]['DAY'].iloc[0] == 'Weekday':
            weekday_event_time = str(individual_grouped_df_list[x]['START_TIME'].iloc[0])
            weekday_event_time = weekday_event_time.replace(':', '')[:-2]
            
        else:
            pass
else:
    weekday_event_time = ''
    

    
for x in individual_grouped_df_list:
    if x['DAY'].iloc[0] == 'Saturday' and num_of_saturday_events == 1:
        vehicle_schedule_text = 'vehicle_schedule|MB-WRIG|00|5|TEST Wrigley #80 Saturday ' + saturday_event_time + '|161|||MBHUTADA|1|0|||||BUS ' + pick[:1] + '-' + pick[1:] + '|' + pick + '-SA|' + pick + '-SA|1||' + pick[:1] + '-SA' + pick[1:] + '|' + pick[:1] + '-SA' + pick[1:] + '|||' + pick + '-SA' + '||BUS ' + pick[:1] + '-' + pick[1:] + '|BUS ' + pick[:1] + '-' + pick[1:] + '|||||0|' + str(random.randint(100000, 200000))
        file_saturday_A = open("file_saturday_A.txt","a")
        file_saturday_A.write(vehicle_schedule_text)
        file_saturday_A.write("\n")
        file_saturday_A.close() 
        
    else:
        vehicle_schedule_text = 'vehicle_schedule|MB-WRIG|00|5|TEST Wrigley #80 Saturday ' + saturday_event_time + '|161|||MBHUTADA|1|0|||||BUS ' + pick[:1] + '-' + pick[1:] + '|' + pick + '-SA|' + pick + '-SA|1||' + pick[:1] + '-SA' + pick[1:] + '|' + pick[:1] + '-SA' + pick[1:] + '|||' + pick + '-SA' + '||BUS ' + pick[:1] + '-' + pick[1:] + '|BUS ' + pick[:1] + '-' + pick[1:] + '|||||0|' + str(random.randint(100000, 200000))
        file_saturday_A = open("file_saturday_A.txt","a")
        file_saturday_A.write(vehicle_schedule_text)
        file_saturday_A.write("\n")
        file_saturday_A.close() 
        break

for x in individual_grouped_df_list:
    if x['DAY'].iloc[0] == 'Sunday' and num_of_sunday_events == 1:
        vehicle_schedule_text = 'vehicle_schedule|MB-WRIG|00|6|TEST Wrigley #80 Sunday ' + sunday_event_time + '|161|||MBHUTADA|1|0|||||BUS ' + pick[:1] + '-' + pick[1:] + '|' + pick + '-SU|' + pick + '-SU|1||' + pick[:1] + '-SU' + pick[1:] + '|' + pick[:1] + '-SU' + pick[1:] + '|||' + pick + '-SU' + '||BUS ' + pick[:1] + '-' + pick[1:] + '|BUS ' + pick[:1] + '-' + pick[1:] + '|||||0|' + str(random.randint(100000, 200000))
        file_sunday_A = open("file_sunday_A.txt","a")
        file_sunday_A.write(vehicle_schedule_text)
        file_sunday_A.write("\n")
        file_sunday_A.close() 
        
    else:
        vehicle_schedule_text = 'vehicle_schedule|MB-WRIG|00|6|TEST Wrigley #80 Sunday ' + sunday_event_time + '|161|||MBHUTADA|1|0|||||BUS ' + pick[:1] + '-' + pick[1:] + '|' + pick + '-SU|' + pick + '-SU|1||' + pick[:1] + '-SU' + pick[1:] + '|' + pick[:1] + '-SU' + pick[1:] + '|||' + pick + '-SU' + '||BUS ' + pick[:1] + '-' + pick[1:] + '|BUS ' + pick[:1] + '-' + pick[1:] + '|||||0|' + str(random.randint(100000, 200000))
        file_sunday_A = open("file_sunday_A.txt","a")
        file_sunday_A.write(vehicle_schedule_text)
        file_sunday_A.write("\n")
        file_sunday_A.close() 
        break

for x in individual_grouped_df_list:
    if x['DAY'].iloc[0] == 'Weekday' and num_of_weekday_events == 1:
        vehicle_schedule_text = 'vehicle_schedule|MB-WRIG|00|0|TEST Wrigley #80 Weekday ' + weekday_event_time + '|161|||MBHUTADA|1|0|||||BUS ' + pick[:1] + '-' + pick[1:] + '|' + pick + '-WK|' + pick + '-WK|1||' + pick[:1] + '-WK' + pick[1:] + '|' + pick[:1] + '-WK' + pick[1:] + '|||' + pick + '-WK' + '||BUS ' + pick[:1] + '-' + pick[1:] + '|BUS ' + pick[:1] + '-' + pick[1:] + '|||||0|' + str(random.randint(100000, 200000))
        file_weekday_A = open("file_weekday_A.txt","a")
        file_weekday_A.write(vehicle_schedule_text)
        file_weekday_A.write("\n")
        file_weekday_A.close()   
        
    else:
        vehicle_schedule_text = 'vehicle_schedule|MB-WRIG|00|0|TEST Wrigley #80 Weekday ' + weekday_event_time + '|161|||MBHUTADA|1|0|||||BUS ' + pick[:1] + '-' + pick[1:] + '|' + pick + '-WK|' + pick + '-WK|1||' + pick[:1] + '-WK' + pick[1:] + '|' + pick[:1] + '-WK' + pick[1:] + '|||' + pick + '-WK' + '||BUS ' + pick[:1] + '-' + pick[1:] + '|BUS ' + pick[:1] + '-' + pick[1:] + '|||||0|' + str(random.randint(100000, 200000))
        file_weekday_A = open("file_weekday_A.txt","a")
        file_weekday_A.write(vehicle_schedule_text)
        file_weekday_A.write("\n")
        file_weekday_A.close()   
        break

In [46]:
for i, q in zip(actual_blocks_df, individual_grouped_df_list):
    
    if q['DAY'].iloc[0] == 'Saturday':
        
        vehicle_unit_number_1 = str(random.randint(20000000, 30000000))
        vehicle_unit_number_2 = str(random.randint(20000000, 30000000))

        vehicle_unit_numbers = [vehicle_unit_number_1, vehicle_unit_number_2]

        vehicle_unit_text_1 = 'vehicle_unit|' + vehicle_unit_number_1 + '||40ft|40ft'
        vehicle_unit_text_2 = 'vehicle_unit|' + vehicle_unit_number_2 + '||40ft|40ft'

        file_saturday_A = open("file_saturday_A.txt","a")
        file_saturday_A.write(vehicle_unit_text_1)
        file_saturday_A.write("\n")
        file_saturday_A.write(vehicle_unit_text_2)
        file_saturday_A.write("\n")
        file_saturday_A.close()  
        
        event_type = q['EVENT_CATEGORY'].iloc[0]
        
        if event_type == 'Cubs':
            event_start_time = str(q['START_TIME'].iloc[0])
            derived_event_start_time = event_start_time.replace(":", "") [:-2]
            event_code = 'CB' + derived_event_start_time
            
        else:
            event_start_time = str(q['START_TIME'].iloc[0])
            derived_event_start_time = event_start_time.replace(":", "") [:-2]
            event_code = 'WF' + derived_event_start_time
            
        
        for j, n in zip(actual_blocks_df[i], range(len(actual_blocks_df[i]))):
            
            block_number = str(random.randint(20000000, 30000000))
            block_text = 'block|' + event_code + '0' + str(n+1) + '||||0|0|40ft|40ft|||F|0|||||' + block_number
            block_vehicle_unit_at_start = 'blk_vehicle_unit_at_start|1|' + vehicle_unit_numbers[n]
            
            file_saturday_B = open("file_saturday_B.txt","a")
            file_saturday_B.write(block_text)
            file_saturday_B.write("\n")
            file_saturday_B.write(block_vehicle_unit_at_start)
            file_saturday_B.write("\n")
            file_saturday_B.close()  
            
            for k in j:
                
                block_activity_number = str(random.randint(200000000, 300000000))
                
                if j.index(k) == 0:
                
                    block_activity_text = 'block_activity|5|' + block_activity_number + '|0|0h00|||'
                    
                elif j.index(k) == len(j) - 1:
                    
                    block_activity_text = 'block_activity|2|' + block_activity_number + '|0|0h00|||'
                
                else:
                    
                    block_activity_text = 'block_activity|6|' + block_activity_number + '|0|0h00|||'
                
                file_saturday_B = open("file_saturday_B.txt","a")
                file_saturday_B.write(block_activity_text)
                file_saturday_B.write("\n")
                file_saturday_B.close()  
                
                trip_number = str(random.randint(80000000, 90000000))
                
                if j.index(k) == 0:
                    
                    e = (datetime.combine(date, k) + timedelta(minutes = 14)).time()
                    if e.second == 0:
                        e_new = str(e)[:-3]
                        
                    elif e.second == 30:
                        e_new = str(e)[:-3] + '+'
                    
                    else:
                        pass
                    
                    if k.second == 0:
                        k_new = str(k)[:-3]
                        
                    elif k.second ==30:
                        k_new = str(k)[:-3] + '+'
                        
                    else:
                        pass
                    
                    pull_out_ref_times_in_str_format = [k_new, e_new]
                        
                    trip_text = 'trip|' + trip_number + '|||1||G--Fg|IrvBlu|G--Fg|IrvBlu||' + k_new + '|' + e_new + '||0h10|2.819|1||||0|||0|0h00|0h00|0|1|0|0|0|0|0|0|1|' + event_code + '|on|F|||1||' + e_new + '|||' + block_activity_number + '|' + block_number
                    file_saturday_C = open("file_saturday_C.txt","a")
                    file_saturday_C.write(trip_text)
                    file_saturday_C.write("\n")
                    file_saturday_C.close()  
                    
                    for l in pull_out_ref_times_in_str_format:
                        if pull_out_ref_times_in_str_format.index(l) == 0:
                            text = 'trip_tp|G--Fg|' + l + '|0h00|0|AS|0.000|0|||0'
                            
                        else:
                            text = 'trip_tp|IrvBlu|' + l + '|0h00|0|AVAS-MMK|2.819|0|||0'
                            
                        file_saturday_C = open("file_saturday_C.txt","a")
                        file_saturday_C.write(text)
                        file_saturday_C.write("\n")
                        file_saturday_C.close()
                    

                elif j.index(k) % 2 == 1 and k != j[len(j) - 2]:
                    
                    refer_trip = EB_runtimes_saturday[(EB_runtimes_saturday['Time'] == k) & (EB_runtimes_saturday['Place'] == 'IrvBlu')]['Trip'].iloc[0]
                    refer_df = EB_runtimes_saturday[(EB_runtimes_saturday['Trip'] == refer_trip) & (EB_runtimes_saturday['Time'] >= k)]
                    ref_dict = dict(zip(refer_df['Place'].tolist(), refer_df['Time'].tolist()))
                    
                    if ref_dict['IrvBlu'].second == 0:
                        k_new = str(k)[:-3]
                        
                    elif ref_dict['IrvBlu'].second == 30:
                        k_new = str(k)[:-3] + '+'
                        
                    else:
                        pass
                    
                    if ref_dict['IrvBwy'].second == 0:
                        e = str(ref_dict['IrvBwy'])[:-3]
                        
                    elif ref_dict['IrvBwy'].second == 30:
                        e = str(ref_dict['IrvBwy'])[:-3] + '+'
                        
                    else:
                        pass
                    
                    trip_text = 'trip|' + trip_number + '|80||0|2|IrvBlu|IrvBwy|IrvBlu|IrvBwy||' + k_new + '|' + e + '|||4.908|1|40ft|||0|||1|0h00|0h00|0|1|0|0|0|0|0|0|1|' + event_code + '|on|F|||0||' + k_new + '|F||' + block_activity_number + '|' + block_number
                    file_saturday_C = open("file_saturday_C.txt","a")
                    file_saturday_C.write(trip_text)
                    file_saturday_C.write("\n")
                    file_saturday_C.close()  
                    
                    for a in ref_dict:
                        
                        if ref_dict[a].second == 30:
                            time_string = str(ref_dict[a])
                            time_string = time_string[:-3] + '+'
                            
                        elif ref_dict[a].second == 0:
                            time_string = str(ref_dict[a])
                            time_string = time_string[:-3]
                        
                        else:
                            pass 
                            
                        text = 'trip_tp|' + a + '|' + time_string + '|0h00|0|' + EB_distances_and_tp_notes[a][1] + '|' + EB_distances_and_tp_notes[a][0] + '|1|||0'
                        file_saturday_C = open("file_saturday_C.txt","a")
                        file_saturday_C.write(text)
                        file_saturday_C.write("\n")
                        file_saturday_C.close()
                    
                    
                elif j.index(k) % 2 == 1 and k == j[len(j) - 2]:
                    
                    refer_trip = EB_runtimes_saturday[(EB_runtimes_saturday['Time'] == k) & (EB_runtimes_saturday['Place'] == 'IrvBlu')]['Trip'].iloc[0]
                    time_points_to_include = ['IrvBlu', 'IrvKim', 'IrvWes', 'IrvAsh', 'SheLs']
                    refer_df = EB_runtimes_saturday[(EB_runtimes_saturday['Trip'] == refer_trip) & (EB_runtimes_saturday['Time'] >= k) & (EB_runtimes_saturday['Place'].isin(time_points_to_include))]
                    ref_dict = dict(zip(refer_df['Place'].tolist(), refer_df['Time'].tolist()))
                    
                    if ref_dict['IrvBlu'].second == 0:
                        k_new = str(k)[:-3]
                        
                    elif ref_dict['IrvBlu'].second == 30:
                        k_new = str(k)[:-3] + '+'
                        
                    else:
                        pass
                    
                    if ref_dict['SheLs'].second == 0:
                        e = str(ref_dict['SheLs'])[:-3]
                        
                    elif ref_dict['SheLs'].second == 30:
                        e = str(ref_dict['SheLs'])[:-3] + '+'
                        
                    else:
                        pass
                    
                    trip_text = 'trip|' + trip_number + '|80||0|2|IrvBlu|SheLs|IrvBlu|SheLs||' + k_new + '|' + e + '|||4.488|1|40ft|||0|||1|0h00|0h00|0|1|0|0|0|0|0|0|1|' + event_code + '|on|F|||0||' + k_new + '|F||' + block_activity_number + '|' + block_number
                    file_saturday_C = open("file_saturday_C.txt","a")
                    file_saturday_C.write(trip_text)
                    file_saturday_C.write("\n")
                    file_saturday_C.close()  
                    
                    for a in ref_dict:
                        
                        if ref_dict[a].second == 30:
                            time_string = str(ref_dict[a])
                            time_string = time_string[:-3] + '+'
                            
                        elif ref_dict[a].second == 0:
                            time_string = str(ref_dict[a])
                            time_string = time_string[:-3]
                        
                        else:
                            pass 
                            
                        text = 'trip_tp|' + a + '|' + time_string + '|0h00|0|' + EB_distances_and_tp_notes[a][1] + '|' + EB_distances_and_tp_notes[a][0] + '|1|||0'
                        file_saturday_C = open("file_saturday_C.txt","a")
                        file_saturday_C.write(text)
                        file_saturday_C.write("\n")
                        file_saturday_C.close()
                    
                    
                elif j.index(k) % 2 == 0 and j.index(k) != 0 and k != j[len(j) - 1]:
                    
                    refer_trip = WB_runtimes_saturday[(WB_runtimes_saturday['Time'] == k) & (WB_runtimes_saturday['Place'] == 'IrvBwy')]['Trip'].iloc[0]
                    time_points_to_include = ['IrvBwy', 'IrvAsh', 'IrvWes', 'IrvKim', 'IrvBlu']
                    refer_df = WB_runtimes_saturday[(WB_runtimes_saturday['Trip'] == refer_trip) & (WB_runtimes_saturday['Time'] >= k) & WB_runtimes_saturday['Place'].isin(time_points_to_include)]
                    ref_dict = dict(zip(refer_df['Place'].tolist(), refer_df['Time'].tolist()))
                    
                    if ref_dict['IrvBwy'].second == 0:
                        k_new = str(k)[:-3]
                        
                    elif ref_dict['IrvBwy'].second == 30:
                        k_new = str(k)[:-3] + '+'
                        
                    else:
                        pass
                        
                    if ref_dict['IrvBlu'].second == 0:
                        e = str(ref_dict['IrvBlu'])[:-3]
                        
                    elif ref_dict['IrvBlu'].second == 30:
                        e = str(ref_dict['IrvBlu'])[:-3] + '+'
                        
                    else:
                        pass
                    
                    trip_text = 'trip|' + trip_number + '|80||0|3|IrvBwy|IrvBlu|IrvBwy|IrvBlu||' + k_new + '|' + e + '|||4.038|1|40ft|||0|||1|0h00|0h00|0|1|0|0|0|0|0|0|1|' + event_code + '|on|F|||0||' + k_new + '|F||' + block_activity_number + '|' + block_number
                    file_saturday_C = open("file_saturday_C.txt","a")
                    file_saturday_C.write(trip_text)
                    file_saturday_C.write("\n")
                    file_saturday_C.close()  
                    
                    for a in ref_dict:
                        
                        if ref_dict[a].second == 30:
                            time_string = str(ref_dict[a])
                            time_string = time_string[:-3] + '+'
                            
                        elif ref_dict[a].second == 0:
                            time_string = str(ref_dict[a])
                            time_string = time_string[:-3]
                        
                        else:
                            pass 
                            
                        text = 'trip_tp|' + a + '|' + time_string + '|0h00|0|' + WB_distances_and_tp_notes[a][1] + '|' + WB_distances_and_tp_notes[a][0] + '|1|||0'
                        file_saturday_C = open("file_saturday_C.txt","a")
                        file_saturday_C.write(text)
                        file_saturday_C.write("\n")
                        file_saturday_C.close()
                        

                elif k == j[len(j) - 1]:
                    
                    e = (datetime.combine(date, k) - timedelta(minutes = 4)).time()
                    if e.second == 0:
                        e_new = str(e)[:-3]
                        
                    elif e.second == 30:
                        e_new = str(e)[:-3] + '+'
                    
                    else:
                        pass
                    
                    if k.second == 0:
                        k_new = str(k)[:-3]
                        
                    elif k.second ==30:
                        k_new = str(k)[:-3] + '+'
                        
                    else:
                        pass
                    
                    pull_in_ref_times_in_str_format = [e_new, k_new]
                        
                    trip_text = 'trip|' + trip_number + '|||3||SheLs|AddHal|SheLs|AddHal||' + e_new + '|' + k_new + '|||0.400|1|40ft|||0|||0|0h00|0h00|1|1|0|0|0|0|0|0|1|' + event_code + '|on||||0||' + e_new + '|F||' + block_activity_number + '|' + block_number
                    file_saturday_C = open("file_saturday_C.txt","a")
                    file_saturday_C.write(trip_text)
                    file_saturday_C.write("\n")
                    file_saturday_C.close()  
                    
                    for l in pull_in_ref_times_in_str_format:
                        if pull_in_ref_times_in_str_format.index(l) == 0:
                            text = 'trip_tp|SheLs|' + l + '|0h00|0||0.000|0|||0'
                            
                        else:
                            text = 'trip_tp|AddHal|' + l + '|0h00|0|' + trip_notes_list[i] + '|0.400|0|||0'
                            
                        file_saturday_C = open("file_saturday_C.txt","a")
                        file_saturday_C.write(text)
                        file_saturday_C.write("\n")
                        file_saturday_C.close()
                    
        
    if q['DAY'].iloc[0] == 'Sunday':
        
        vehicle_unit_number_1 = str(random.randint(20000000, 30000000))
        vehicle_unit_number_2 = str(random.randint(20000000, 30000000))

        vehicle_unit_numbers = [vehicle_unit_number_1, vehicle_unit_number_2]

        vehicle_unit_text_1 = 'vehicle_unit|' + vehicle_unit_number_1 + '||40ft|40ft'
        vehicle_unit_text_2 = 'vehicle_unit|' + vehicle_unit_number_2 + '||40ft|40ft'

        file_sunday_A = open("file_sunday_A.txt","a")
        file_sunday_A.write(vehicle_unit_text_1)
        file_sunday_A.write("\n")
        file_sunday_A.write(vehicle_unit_text_2)
        file_sunday_A.write("\n")
        file_sunday_A.close()  
        
        event_type = q['EVENT_CATEGORY'].iloc[0]
        
        if event_type == 'Cubs':
            event_start_time = str(q['START_TIME'].iloc[0])
            derived_event_start_time = event_start_time.replace(":", "") [:-2]
            event_code = 'CB' + derived_event_start_time
            
        else:
            event_start_time = str(q['START_TIME'].iloc[0])
            derived_event_start_time = event_start_time.replace(":", "") [:-2]
            event_code = 'WF' + derived_event_start_time
            
        
        for j, n in zip(actual_blocks_df[i], range(len(actual_blocks_df[i]))):
            
            block_number = str(random.randint(20000000, 30000000))
            block_text = 'block|' + event_code + '0' + str(n+1) + '||||0|0|40ft|40ft|||F|0|||||' + block_number
            block_vehicle_unit_at_start = 'blk_vehicle_unit_at_start|1|' + vehicle_unit_numbers[n]
            
            file_sunday_B = open("file_sunday_B.txt","a")
            file_sunday_B.write(block_text)
            file_sunday_B.write("\n")
            file_sunday_B.write(block_vehicle_unit_at_start)
            file_sunday_B.write("\n")
            file_sunday_B.close()  
            
            for k in j:
                
                block_activity_number = str(random.randint(200000000, 300000000))
                
                if j.index(k) == 0:
                
                    block_activity_text = 'block_activity|5|' + block_activity_number + '|0|0h00|||'
                    
                elif j.index(k) == len(j) - 1:
                    
                    block_activity_text = 'block_activity|2|' + block_activity_number + '|0|0h00|||'
                
                else:
                    
                    block_activity_text = 'block_activity|6|' + block_activity_number + '|0|0h00|||'
                
                file_sunday_B = open("file_sunday_B.txt","a")
                file_sunday_B.write(block_activity_text)
                file_sunday_B.write("\n")
                file_sunday_B.close()  
                
                trip_number = str(random.randint(80000000, 90000000))
                
                if j.index(k) == 0:
                    
                    e = (datetime.combine(date, k) + timedelta(minutes = 14)).time()
                    if e.second == 0:
                        e_new = str(e)[:-3]
                        
                    elif e.second == 30:
                        e_new = str(e)[:-3] + '+'
                    
                    else:
                        pass
                    
                    if k.second == 0:
                        k_new = str(k)[:-3]
                        
                    elif k.second ==30:
                        k_new = str(k)[:-3] + '+'
                        
                    else:
                        pass
                    
                    pull_out_ref_times_in_str_format = [k_new, e_new]
                        
                    trip_text = 'trip|' + trip_number + '|||1||G--Fg|IrvBlu|G--Fg|IrvBlu||' + k_new + '|' + e_new + '||0h10|3.300|1||||0|||0|0h00|0h00|0|1|1|0|0|0|0|0|0|' + event_code + '|on|F|||1||' + e_new + '|||' + block_activity_number + '|' + block_number
                    file_sunday_C = open("file_sunday_C.txt","a")
                    file_sunday_C.write(trip_text)
                    file_sunday_C.write("\n")
                    file_sunday_C.close()  
                    
                    for l in pull_out_ref_times_in_str_format:
                        if pull_out_ref_times_in_str_format.index(l) == 0:
                            text = 'trip_tp|G--Fg|' + l + '|0h00|0|AS|0.000|0|||0'
                            
                        else:
                            text = 'trip_tp|IrvBlu|' + l + '|0h00|0|AVAS-MMK|3.300|0|||0'
                            
                        file_sunday_C = open("file_sunday_C.txt","a")
                        file_sunday_C.write(text)
                        file_sunday_C.write("\n")
                        file_sunday_C.close()
                    

                elif j.index(k) % 2 == 1 and k != j[len(j) - 2]:
                    
                    refer_trip = EB_runtimes_sunday[(EB_runtimes_sunday['Time'] == k) & (EB_runtimes_sunday['Place'] == 'IrvBlu')]['Trip'].iloc[0]
                    refer_df = EB_runtimes_sunday[(EB_runtimes_sunday['Trip'] == refer_trip) & (EB_runtimes_sunday['Time'] >= k)]
                    ref_dict = dict(zip(refer_df['Place'].tolist(), refer_df['Time'].tolist()))
                    
                    if ref_dict['IrvBlu'].second == 0:
                        k_new = str(k)[:-3]
                        
                    elif ref_dict['IrvBlu'].second == 30:
                        k_new = str(k)[:-3] + '+'
                        
                    else:
                        pass
                    
                    if ref_dict['IrvBwy'].second == 0:
                        e = str(ref_dict['IrvBwy'])[:-3]
                        
                    elif ref_dict['IrvBwy'].second == 30:
                        e = str(ref_dict['IrvBwy'])[:-3] + '+'
                        
                    else:
                        pass
                    
                    trip_text = 'trip|' + trip_number + '|80||0|2|IrvBlu|IrvBwy|IrvBlu|IrvBwy||' + k_new + '|' + e + '|||4.908|1|40ft|||0|||1|0h00|0h00|0|1|1|0|0|0|0|0|0|' + event_code + '|on|F|||0||' + k_new + '|F||' + block_activity_number + '|' + block_number
                    file_sunday_C = open("file_sunday_C.txt","a")
                    file_sunday_C.write(trip_text)
                    file_sunday_C.write("\n")
                    file_sunday_C.close()  
                    
                    for a in ref_dict:
                        
                        if ref_dict[a].second == 30:
                            time_string = str(ref_dict[a])
                            time_string = time_string[:-3] + '+'
                            
                        elif ref_dict[a].second == 0:
                            time_string = str(ref_dict[a])
                            time_string = time_string[:-3]
                        
                        else:
                            pass 
                            
                        text = 'trip_tp|' + a + '|' + time_string + '|0h00|0|' + EB_distances_and_tp_notes[a][1] + '|' + EB_distances_and_tp_notes[a][0] + '|1|||0'
                        file_sunday_C = open("file_sunday_C.txt","a")
                        file_sunday_C.write(text)
                        file_sunday_C.write("\n")
                        file_sunday_C.close()
                    
                    
                elif j.index(k) % 2 == 1 and k == j[len(j) - 2]:
                    
                    refer_trip = EB_runtimes_sunday[(EB_runtimes_sunday['Time'] == k) & (EB_runtimes_sunday['Place'] == 'IrvBlu')]['Trip'].iloc[0]
                    time_points_to_include = ['IrvBlu', 'IrvKim', 'IrvWes', 'IrvAsh', 'SheLs']
                    refer_df = EB_runtimes_sunday[(EB_runtimes_sunday['Trip'] == refer_trip) & (EB_runtimes_sunday['Time'] >= k) & (EB_runtimes_sunday['Place'].isin(time_points_to_include))]
                    ref_dict = dict(zip(refer_df['Place'].tolist(), refer_df['Time'].tolist()))
                    
                    if ref_dict['IrvBlu'].second == 0:
                        k_new = str(k)[:-3]
                        
                    elif ref_dict['IrvBlu'].second == 30:
                        k_new = str(k)[:-3] + '+'
                        
                    else:
                        pass
                    
                    if ref_dict['SheLs'].second == 0:
                        e = str(ref_dict['SheLs'])[:-3]
                        
                    elif ref_dict['SheLs'].second == 30:
                        e = str(ref_dict['SheLs'])[:-3] + '+'
                        
                    else:
                        pass
                    
                    trip_text = 'trip|' + trip_number + '|80||0|2|IrvBlu|SheLs|IrvBlu|SheLs||' + k_new + '|' + e + '|||4.488|1|40ft|||0|||1|0h00|0h00|0|1|1|0|0|0|0|0|0|' + event_code + '|on|F|||0||' + k_new + '|F||' + block_activity_number + '|' + block_number
                    file_sunday_C = open("file_sunday_C.txt","a")
                    file_sunday_C.write(trip_text)
                    file_sunday_C.write("\n")
                    file_sunday_C.close()  
                    
                    for a in ref_dict:
                        
                        if ref_dict[a].second == 30:
                            time_string = str(ref_dict[a])
                            time_string = time_string[:-3] + '+'
                            
                        elif ref_dict[a].second == 0:
                            time_string = str(ref_dict[a])
                            time_string = time_string[:-3]
                        
                        else:
                            pass 
                            
                        text = 'trip_tp|' + a + '|' + time_string + '|0h00|0|' + EB_distances_and_tp_notes[a][1] + '|' + EB_distances_and_tp_notes[a][0] + '|1|||0'
                        file_sunday_C = open("file_sunday_C.txt","a")
                        file_sunday_C.write(text)
                        file_sunday_C.write("\n")
                        file_sunday_C.close()
                    
                    
                elif j.index(k) % 2 == 0 and j.index(k) != 0 and k != j[len(j) - 1]:
                    
                    refer_trip = WB_runtimes_sunday[(WB_runtimes_sunday['Time'] == k) & (WB_runtimes_sunday['Place'] == 'IrvBwy')]['Trip'].iloc[0]
                    time_points_to_include = ['IrvBwy', 'IrvAsh', 'IrvWes', 'IrvKim', 'IrvBlu']
                    refer_df = WB_runtimes_sunday[(WB_runtimes_sunday['Trip'] == refer_trip) & (WB_runtimes_sunday['Time'] >= k) & WB_runtimes_sunday['Place'].isin(time_points_to_include)]
                    ref_dict = dict(zip(refer_df['Place'].tolist(), refer_df['Time'].tolist()))
                    
                    if ref_dict['IrvBwy'].second == 0:
                        k_new = str(k)[:-3]
                        
                    elif ref_dict['IrvBwy'].second == 30:
                        k_new = str(k)[:-3] + '+'
                        
                    else:
                        pass
                        
                    if ref_dict['IrvBlu'].second == 0:
                        e = str(ref_dict['IrvBlu'])[:-3]
                        
                    elif ref_dict['IrvBlu'].second == 30:
                        e = str(ref_dict['IrvBlu'])[:-3] + '+'
                        
                    else:
                        pass
                    
                    trip_text = 'trip|' + trip_number + '|80||0|3|IrvBwy|IrvBlu|IrvBwy|IrvBlu||' + k_new + '|' + e + '|||4.038|1|40ft|||0|||1|0h00|0h00|0|1|1|0|0|0|0|0|0|' + event_code + '|on|F|||0||' + k_new + '|F||' + block_activity_number + '|' + block_number
                    file_sunday_C = open("file_sunday_C.txt","a")
                    file_sunday_C.write(trip_text)
                    file_sunday_C.write("\n")
                    file_sunday_C.close()  
                    
                    for a in ref_dict:
                        
                        if ref_dict[a].second == 30:
                            time_string = str(ref_dict[a])
                            time_string = time_string[:-3] + '+'
                            
                        elif ref_dict[a].second == 0:
                            time_string = str(ref_dict[a])
                            time_string = time_string[:-3]
                        
                        else:
                            pass 
                            
                        text = 'trip_tp|' + a + '|' + time_string + '|0h00|0|' + WB_distances_and_tp_notes[a][1] + '|' + WB_distances_and_tp_notes[a][0] + '|1|||0'
                        file_sunday_C = open("file_sunday_C.txt","a")
                        file_sunday_C.write(text)
                        file_sunday_C.write("\n")
                        file_sunday_C.close()
                        

                elif k == j[len(j) - 1]:
                    
                    e = (datetime.combine(date, k) - timedelta(minutes = 4)).time()
                    if e.second == 0:
                        e_new = str(e)[:-3]
                        
                    elif e.second == 30:
                        e_new = str(e)[:-3] + '+'
                    
                    else:
                        pass
                    
                    if k.second == 0:
                        k_new = str(k)[:-3]
                        
                    elif k.second ==30:
                        k_new = str(k)[:-3] + '+'
                        
                    else:
                        pass
                    
                    pull_in_ref_times_in_str_format = [e_new, k_new]
                        
                    trip_text = 'trip|' + trip_number + '|||3||SheLs|AddHal|SheLs|AddHal||' + e_new + '|' + k_new + '||||1||||0|||0|0h00|0h00|1|1|1|0|0|0|0|0|0|' + event_code + '|on||||0||' + e_new + '|||' + block_activity_number + '|' + block_number
                    file_sunday_C = open("file_sunday_C.txt","a")
                    file_sunday_C.write(trip_text)
                    file_sunday_C.write("\n")
                    file_sunday_C.close()  
                    
                    for l in pull_in_ref_times_in_str_format:
                        if pull_in_ref_times_in_str_format.index(l) == 0:
                            text = 'trip_tp|SheLs|' + l + '|0h00|0||0.000|0|||0'
                            
                        else:
                            text = 'trip_tp|AddHal|' + l + '|0h00|0|' + trip_notes_list[i] + '||0|||0'
                            
                        file_sunday_C = open("file_sunday_C.txt","a")
                        file_sunday_C.write(text)
                        file_sunday_C.write("\n")
                        file_sunday_C.close()

    
    if q['DAY'].iloc[0] == 'Weekday':    
        
        vehicle_unit_number_1 = str(random.randint(20000000, 30000000))
        vehicle_unit_number_2 = str(random.randint(20000000, 30000000))

        vehicle_unit_numbers = [vehicle_unit_number_1, vehicle_unit_number_2]

        vehicle_unit_text_1 = 'vehicle_unit|' + vehicle_unit_number_1 + '||40ft|40ft'
        vehicle_unit_text_2 = 'vehicle_unit|' + vehicle_unit_number_2 + '||40ft|40ft'

        file_weekday_A = open("file_weekday_A.txt","a")
        file_weekday_A.write(vehicle_unit_text_1)
        file_weekday_A.write("\n")
        file_weekday_A.write(vehicle_unit_text_2)
        file_weekday_A.write("\n")
        file_weekday_A.close()  
        
        event_type = q['EVENT_CATEGORY'].iloc[0]
        
        if event_type == 'Cubs':
            event_start_time = str(q['START_TIME'].iloc[0])
            derived_event_start_time = event_start_time.replace(":", "") [:-2]
            event_code = 'CB' + derived_event_start_time
            
        else:
            event_start_time = str(q['START_TIME'].iloc[0])
            derived_event_start_time = event_start_time.replace(":", "") [:-2]
            event_code = 'WF' + derived_event_start_time
            
        
        for j, n in zip(actual_blocks_df[i], range(len(actual_blocks_df[i]))):
            
            block_number = str(random.randint(20000000, 30000000))
            block_text = 'block|' + event_code + '0' + str(n+1) + '||||0|0|40ft|40ft|||F|0|||||' + block_number
            block_vehicle_unit_at_start = 'blk_vehicle_unit_at_start|1|' + vehicle_unit_numbers[n]
            
            file_weekday_B = open("file_weekday_B.txt","a")
            file_weekday_B.write(block_text)
            file_weekday_B.write("\n")
            file_weekday_B.write(block_vehicle_unit_at_start)
            file_weekday_B.write("\n")
            file_weekday_B.close()  
            
            for k in j:
                
                block_activity_number = str(random.randint(200000000, 300000000))
                
                if j.index(k) == 0:
                
                    block_activity_text = 'block_activity|5|' + block_activity_number + '|0|0h00|||'
                    
                elif j.index(k) == len(j) - 1:
                    
                    block_activity_text = 'block_activity|2|' + block_activity_number + '|0|0h00|||'
                
                else:
                    
                    block_activity_text = 'block_activity|6|' + block_activity_number + '|0|0h00|||'
                
                file_weekday_B = open("file_weekday_B.txt","a")
                file_weekday_B.write(block_activity_text)
                file_weekday_B.write("\n")
                file_weekday_B.close()  
                
                trip_number = str(random.randint(80000000, 90000000))
                
                if j.index(k) == 0:
                    
                    e = (datetime.combine(date, k) + timedelta(minutes = 14)).time()
                    if e.second == 0:
                        e_new = str(e)[:-3]
                        
                    elif e.second == 30:
                        e_new = str(e)[:-3] + '+'
                    
                    else:
                        pass
                    
                    if k.second == 0:
                        k_new = str(k)[:-3]
                        
                    elif k.second ==30:
                        k_new = str(k)[:-3] + '+'
                        
                    else:
                        pass
                    
                    pull_out_ref_times_in_str_format = [k_new, e_new]
                        
                    trip_text = 'trip|' + trip_number + '|||1||G--Fg|IrvBlu|G--Fg|IrvBlu||' + k_new + '|' + e_new + '||0h10|2.819|1||||0|||0|0h00|0h00|0|1|0|1|1|1|1|1|0|' + event_code + '|on|F|||1||' + e_new + '|||' + block_activity_number + '|' + block_number
                    file_weekday_C = open("file_weekday_C.txt","a")
                    file_weekday_C.write(trip_text)
                    file_weekday_C.write("\n")
                    file_weekday_C.close()  
                    
                    for l in pull_out_ref_times_in_str_format:
                        if pull_out_ref_times_in_str_format.index(l) == 0:
                            text = 'trip_tp|G--Fg|' + l + '|0h00|0|AS|0.000|0|||0'
                            
                        else:
                            text = 'trip_tp|IrvBlu|' + l + '|0h00|0|AVAS-MMK|2.819|0|||0'
                            
                        file_weekday_C = open("file_weekday_C.txt","a")
                        file_weekday_C.write(text)
                        file_weekday_C.write("\n")
                        file_weekday_C.close()
                    

                elif j.index(k) % 2 == 1 and k != j[len(j) - 2]:
                    
                    refer_trip = EB_runtimes_weekday[(EB_runtimes_weekday['Time'] == k) & (EB_runtimes_weekday['Place'] == 'IrvBlu')]['Trip'].iloc[0]
                    refer_df = EB_runtimes_weekday[(EB_runtimes_weekday['Trip'] == refer_trip) & (EB_runtimes_weekday['Time'] >= k)]
                    ref_dict = dict(zip(refer_df['Place'].tolist(), refer_df['Time'].tolist()))
                    
                    if ref_dict['IrvBlu'].second == 0:
                        k_new = str(k)[:-3]
                        
                    elif ref_dict['IrvBlu'].second == 30:
                        k_new = str(k)[:-3] + '+'
                        
                    else:
                        pass
                    
                    if ref_dict['IrvBwy'].second == 0:
                        e = str(ref_dict['IrvBwy'])[:-3]
                        
                    elif ref_dict['IrvBwy'].second == 30:
                        e = str(ref_dict['IrvBwy'])[:-3] + '+'
                        
                    else:
                        pass
                    
                    trip_text = 'trip|' + trip_number + '|80||0|2|IrvBlu|IrvBwy|IrvBlu|IrvBwy||' + k_new + '|' + e + '|||4.908|1|40ft|||0|||1|0h00|0h00|0|1|0|1|1|1|1|1|0|' + event_code + '|on|F|||0||' + k_new + '|F||' + block_activity_number + '|' + block_number
                    file_weekday_C = open("file_weekday_C.txt","a")
                    file_weekday_C.write(trip_text)
                    file_weekday_C.write("\n")
                    file_weekday_C.close()  
                    
                    for a in ref_dict:
                        
                        if ref_dict[a].second == 30:
                            time_string = str(ref_dict[a])
                            time_string = time_string[:-3] + '+'
                            
                        elif ref_dict[a].second == 0:
                            time_string = str(ref_dict[a])
                            time_string = time_string[:-3]
                        
                        else:
                            pass 
                            
                        text = 'trip_tp|' + a + '|' + time_string + '|0h00|0|' + EB_distances_and_tp_notes[a][1] + '|' + EB_distances_and_tp_notes[a][0] + '|1|||0'
                        file_weekday_C = open("file_weekday_C.txt","a")
                        file_weekday_C.write(text)
                        file_weekday_C.write("\n")
                        file_weekday_C.close()
                    
                    
                elif j.index(k) % 2 == 1 and k == j[len(j) - 2]:
                    
                    refer_trip = EB_runtimes_weekday[(EB_runtimes_weekday['Time'] == k) & (EB_runtimes_weekday['Place'] == 'IrvBlu')]['Trip'].iloc[0]
                    time_points_to_include = ['IrvBlu', 'IrvKim', 'IrvWes', 'IrvAsh', 'SheLs']
                    refer_df = EB_runtimes_weekday[(EB_runtimes_weekday['Trip'] == refer_trip) & (EB_runtimes_weekday['Time'] >= k) & (EB_runtimes_weekday['Place'].isin(time_points_to_include))]
                    ref_dict = dict(zip(refer_df['Place'].tolist(), refer_df['Time'].tolist()))
                    
                    if ref_dict['IrvBlu'].second == 0:
                        k_new = str(k)[:-3]
                        
                    elif ref_dict['IrvBlu'].second == 30:
                        k_new = str(k)[:-3] + '+'
                        
                    else:
                        pass
                    
                    if ref_dict['SheLs'].second == 0:
                        e = str(ref_dict['SheLs'])[:-3]
                        
                    elif ref_dict['SheLs'].second == 30:
                        e = str(ref_dict['SheLs'])[:-3] + '+'
                        
                    else:
                        pass
                    
                    trip_text = 'trip|' + trip_number + '|80||0|2|IrvBlu|SheLs|IrvBlu|SheLs||' + k_new + '|' + e + '|||4.488|1|40ft|||0|||1|0h00|0h00|0|1|0|1|1|1|1|1|0|' + event_code + '|on|F|||0||' + k_new + '|F||' + block_activity_number + '|' + block_number
                    file_weekday_C = open("file_weekday_C.txt","a")
                    file_weekday_C.write(trip_text)
                    file_weekday_C.write("\n")
                    file_weekday_C.close()  
                    
                    for a in ref_dict:
                        
                        if ref_dict[a].second == 30:
                            time_string = str(ref_dict[a])
                            time_string = time_string[:-3] + '+'
                            
                        elif ref_dict[a].second == 0:
                            time_string = str(ref_dict[a])
                            time_string = time_string[:-3]
                        
                        else:
                            pass 
                            
                        text = 'trip_tp|' + a + '|' + time_string + '|0h00|0|' + EB_distances_and_tp_notes[a][1] + '|' + EB_distances_and_tp_notes[a][0] + '|1|||0'
                        file_weekday_C = open("file_weekday_C.txt","a")
                        file_weekday_C.write(text)
                        file_weekday_C.write("\n")
                        file_weekday_C.close()
                    
                    
                elif j.index(k) % 2 == 0 and j.index(k) != 0 and k != j[len(j) - 1]:
                    
                    refer_trip = WB_runtimes_weekday[(WB_runtimes_weekday['Time'] == k) & (WB_runtimes_weekday['Place'] == 'IrvBwy')]['Trip'].iloc[0]
                    time_points_to_include = ['IrvBwy', 'IrvAsh', 'IrvWes', 'IrvKim', 'IrvBlu']
                    refer_df = WB_runtimes_weekday[(WB_runtimes_weekday['Trip'] == refer_trip) & (WB_runtimes_weekday['Time'] >= k) & WB_runtimes_weekday['Place'].isin(time_points_to_include)]
                    ref_dict = dict(zip(refer_df['Place'].tolist(), refer_df['Time'].tolist()))
                    
                    if ref_dict['IrvBwy'].second == 0:
                        k_new = str(k)[:-3]
                        
                    elif ref_dict['IrvBwy'].second == 30:
                        k_new = str(k)[:-3] + '+'
                        
                    else:
                        pass
                        
                    if ref_dict['IrvBlu'].second == 0:
                        e = str(ref_dict['IrvBlu'])[:-3]
                        
                    elif ref_dict['IrvBlu'].second == 30:
                        e = str(ref_dict['IrvBlu'])[:-3] + '+'
                        
                    else:
                        pass
                    
                    trip_text = 'trip|' + trip_number + '|80||0|3|IrvBwy|IrvBlu|IrvBwy|IrvBlu||' + k_new + '|' + e + '|||4.038|1|40ft|||0|||1|0h00|0h00|0|1|0|1|1|1|1|1|0|' + event_code + '|on|F|||0||' + k_new + '|F||' + block_activity_number + '|' + block_number
                    file_weekday_C = open("file_weekday_C.txt","a")
                    file_weekday_C.write(trip_text)
                    file_weekday_C.write("\n")
                    file_weekday_C.close()  
                    
                    for a in ref_dict:
                        
                        if ref_dict[a].second == 30:
                            time_string = str(ref_dict[a])
                            time_string = time_string[:-3] + '+'
                            
                        elif ref_dict[a].second == 0:
                            time_string = str(ref_dict[a])
                            time_string = time_string[:-3]
                        
                        else:
                            pass 
                            
                        text = 'trip_tp|' + a + '|' + time_string + '|0h00|0|' + WB_distances_and_tp_notes[a][1] + '|' + WB_distances_and_tp_notes[a][0] + '|1|||0'
                        file_weekday_C = open("file_weekday_C.txt","a")
                        file_weekday_C.write(text)
                        file_weekday_C.write("\n")
                        file_weekday_C.close()
                        

                elif k == j[len(j) - 1]:
                    
                    e = (datetime.combine(date, k) - timedelta(minutes = 4)).time()
                    if e.second == 0:
                        e_new = str(e)[:-3]
                        
                    elif e.second == 30:
                        e_new = str(e)[:-3] + '+'
                    
                    else:
                        pass
                    
                    if k.second == 0:
                        k_new = str(k)[:-3]
                        
                    elif k.second ==30:
                        k_new = str(k)[:-3] + '+'
                        
                    else:
                        pass
                    
                    pull_in_ref_times_in_str_format = [e_new, k_new]
                        
                    trip_text = 'trip|' + trip_number + '|||3||SheLs|AddHal|SheLs|AddHal||' + e_new + '|' + k_new + '|||0.400|1|40ft|||0|||0|0h00|0h00|1|1|0|1|1|1|1|1|0|' + event_code + '|on||||0||' + e_new + '|F||' + block_activity_number + '|' + block_number
                    file_weekday_C = open("file_weekday_C.txt","a")
                    file_weekday_C.write(trip_text)
                    file_weekday_C.write("\n")
                    file_weekday_C.close()  
                    
                    for l in pull_in_ref_times_in_str_format:
                        if pull_in_ref_times_in_str_format.index(l) == 0:
                            text = 'trip_tp|SheLs|' + l + '|0h00|0||0.000|0|||0'
                            
                        else:
                            text = 'trip_tp|AddHal|' + l + '|0h00|0|' + trip_notes_list[i] + '|0.400|0|||0'
                            
                        file_weekday_C = open("file_weekday_C.txt","a")
                        file_weekday_C.write(text)
                        file_weekday_C.write("\n")
                        file_weekday_C.close()

        
        

filenames_saturday = ['file_saturday_A.txt', 'file_saturday_B.txt', 'file_saturday_C.txt']
 
with open('file_saturday_main.txt', 'w') as outfile:
 
    for name in filenames_saturday:
 
        with open(name) as infile:
 
            outfile.write(infile.read())
    



filenames_sunday = ['file_sunday_A.txt', 'file_sunday_B.txt', 'file_sunday_C.txt']
 
with open('file_sunday_main.txt', 'w') as outfile:
 
    for name in filenames_sunday:
 
        with open(name) as infile:
 
            outfile.write(infile.read())
    
    

    
filenames_weekday = ['file_weekday_A.txt', 'file_weekday_B.txt', 'file_weekday_C.txt']
 
with open('file_weekday_main.txt', 'w') as outfile:
 
    for name in filenames_weekday:
 
        with open(name) as infile:
 
            outfile.write(infile.read())